In [4]:
import os
%pwd

'c:\\Users\\lpaes\\projects\\Xray_img_detection\\research'

In [5]:
os.chdir('../')
%pwd

'c:\\Users\\lpaes\\projects\\Xray_img_detection'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallBacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [15]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PrepareCallBacksConfig:
        config = self.config.prepare_callbacks

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallBacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [18]:
import os
import time
import tensorflow as tf
import urllib.request as request

#6. Create the component 

class PrepareCallBack:
    def __init__(self, config: PrepareCallBacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_ckpt_callbacks,
            self._create_ckpt_callbacks,
        ]

In [19]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallBack(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-07-21 15:21:16,290: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-21 15:21:16,294: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-21 15:21:16,296: INFO: common: created directory at: artifacts]
[2023-07-21 15:21:16,297: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-21 15:21:16,299: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


AttributeError: 'PrepareCallBack' object has no attribute '_creat_ckpt_callbacks'